# Create Space-time Grid Stacks for Met Variables

#### Table of Contents

1.  <a href="#intro_met">Introduction</a> <br>
2.  <a href="#specify_basin">Specify a River Basin</a> <br>
3.  <a href="#GLDAS_to_RTS">Create GLDAS Grid Stack</a> <br>
4.  <a href="#GPM_to_RTS">Create GPM Grid Stack</a> <br>
5.  <a href="#CHIRPS_to_RTS">Create CHIRPS Grid Stack</a> <br>

<!-- Hyperlink IDs must be unique to work in Jupyter Lab when
there are multiple notebooks open. Hence the trailing letter. -->

## Introduction <a id="intro_met"></a>

<b>Note:</b> This capability has been mostly delegated to the MINT data transformation team, since downloading all of the required files can take hours to days.

However, assuming all required files have already been downloaded to your computer it can also be performed using TopoFlow's <b>regrid</b> utility.  The specific function for this is called: <b>create_rts_from_nc_files()</b>.

For the [<b>MINT Project</b>](http://mint-project.info/), we have used three different
<b>global</b>, remote-sensing based products for rainfall, called:
<b>GPM</b>, <b>GLDAS</b>, and <b>CHIRPS</b>.

In addition to space-time rainfall rates, some of the TopoFlow components can make use of other meteorological variables such as:  air temperature, soil temperature, relative humidity, surface wind speed, shortware radiation and longwave radiation. 
The <b>create_rts_from_nc_files()</b> function has a <b>var_name</b> argument and other settings to accommodate these additional variables.

### Import the TopoFlow regrid Utility

In [5]:
import os
from topoflow.utils import regrid

## Specify a River Basin  <a id="specify_basin"></a>

In [6]:
tf_data_dir = '/Users/peckhams/TF_Data/'

site_prefix = 'Baro-Gam_60sec'
# case_prefix  = 'Test1'
if (site_prefix == 'Baro-Gam_60sec'):
    #--------------------------------------------
    # DEM information for Baro River, Ethiopia
    #--------------------------------------------
    DEM_xres_sec = 60.0  # [arcseconds]
    DEM_yres_sec = 60.0
    DEM_ncols    = 134
    DEM_nrows    = 129
    minlat = 7.362083333332    # (south)
    maxlat = 9.503749999999    # (north)
    minlon = 34.221249999999   # (west)
    maxlon = 36.450416666666   # (east)
    DEM_bounds = [minlon, minlat, maxlon, maxlat]

    gldas_dir   = tf_data_dir + 'GLDAS_Rain_Baro_2015-10_to_2018-10/'
    gpm_dir     = tf_data_dir + 'GPM_Rain_Ethiopia_2015-10_to_2018-10/'
    chirps_dir  = tf_data_dir + 'CHIRPS_Rain_Baro_2015-10_to_2018-10/'
    chirps_dir2 = tf_data_dir + 'CHIRPS_Rain_Africa_6hr_2015-10_to_2020-06/'
    
    # max_river_width = 140.0  # [meters]  from Google Maps or Google Earth
    # A_out_km2 = 23567.7  # total contributing area at basin outlet [km2]
    # Qbase_out = 40.0     # estimated baseflow discharge at basin outlet [m^3 / s]
    # max_bankfull_depth = 8.0   # [meters]    # from literature or data


## Create GLDAS Grid Stack  <a id="GLDAS_to_RTS"></a>

[<b>Version 2.1 of the GLDAS Noah Land Surface Model L4</b>](https://disc.gsfc.nasa.gov/datasets/GLDAS_NOAH025_3H_2.1/summary?keywords=GLDAS)
has a spatial resolution of <b>0.25 x 0.25 degrees</b> (900 x 900 arcseconds)
and a temporal resolution of <b>3 hours</b> (180 minutes).
This data is available for dates between: <b>2000-01-01</b> and <b>2020-02-29</b>.
This is the highest spatial and temporal resolution available for GLDAS data.

GLDAS has multiple space-time products, including <b>precipitation</b>, <b>air temperature</b> and <b>evaporation rate</b>.

In [ ]:
#-----------------------------------------
# Some GLDAS variable names;  Choose one
#-----------------------------------------
var_name = 'Rainf_tavg'
# var_name = 'Tair_f_inst'   # (air temperature)
# var_name = 'Evap_tavg'
# var_name = 'Swnet_tavg'
# var_name = 'Lwnet_tavg'
# var_name = 'Qsb_acc'     # (baseflow groundwater runoff)
# var_name = 'SoilMoi0_10cm_inst'  # (soil moisture content)
# var_name = 'Albedo_inst'
# var_name = 'Wind_f_inst'

if (var_name == 'Rainf_tavg'):
    rts_file = gldas_dir + '__GLDAS_Rain_Baro_2015-10_to_2018-10.rts'
if (var_name == 'Tair_f_inst'):
    rts_file = gldas_dir + '__GLDAS_Tair_Baro_2015-10_to_2018-10.rts'
if (var_name == 'Evap_tavg'):
    rts_file = gldas_dir + '__GLDAS_Evap_Baro_2015-10_to_2018-10.rts'

NC4 = True
GPM = False   # (so GLDAS=True)
resample_algo = 'bilinear'
## resample_algo = 'nearest'

# GPM variable name
# var_name = 'precipitationCal' # [mmph]
            
os.chdir( gldas_dir )
regrid.create_rts_from_nc_files( rts_file=rts_file, NC4=NC4,
           var_name=var_name, GPM=GPM, resample_algo=resample_algo,
           VERBOSE=False, DEM_bounds=DEM_bounds,
           DEM_xres_sec=DEM_xres_sec, DEM_yres_sec=DEM_yres_sec,
           DEM_ncols=DEM_ncols, DEM_nrows=DEM_nrows )


## Create GPM Grid Stack  <a id="GPM_to_RTS"></a>

[<b>Version 06 of the GPM GPM IMERG Final Precipitation L3 </b>](https://disc.gsfc.nasa.gov/datasets/GPM_3IMERGHH_06/summary?keywords=GPM)
has a spatial resolution of <b>0.1 x 0.1 degrees</b> (360 x 360 arcseconds)
and a temporal resolution of <b>30 minutes</b> (1800 seconds). 
This data is available for dates between: <b>2000-06-01</b> and <b>2020-01-31</b>.
This is the highest spatial and temporal resolution available for GPM data.


In [7]:
rts_file = gpm_dir + '__GPM_Rain_Baro_2015-10_to_2018-10.rts'
# GPM variable name
var_name = 'precipitationCal'  # [mmph]

NC4 = True
GPM = True
resample_algo = 'bilinear'
## resample_algo = 'nearest'
          
os.chdir( gpm_dir )
regrid.create_rts_from_nc_files( rts_file=rts_file, NC4=NC4,
           var_name=var_name, GPM=GPM, resample_algo=resample_algo,
           VERBOSE=False, DEM_bounds=DEM_bounds,
           DEM_xres_sec=DEM_xres_sec, DEM_yres_sec=DEM_yres_sec,
           DEM_ncols=DEM_ncols, DEM_nrows=DEM_nrows )


FileNotFoundError: [Errno 2] No such file or directory: '/Users/peckhams/TF_Data/GPM_Rain_Baro_2015-10_to_2018-10/'

## Create CHIRPS Grid Stack  <a id="CHIRPS_to_RTS"></a>

Here we use the CHIRPS Africa dataset, and a function in the TopoFlow <b>regrid</b>
utility called <b>create_rts_from_chirps_files()</b>.  This function is currently
hardwired to use the CHIRPS Africa dataset, which has the following georeferencing
information.

    #-----------------------------------    
    # CHIRPS Africa bounding box, etc.
    #-----------------------------------
    # chirps_minlat = -40.05
    # chirps_maxlat = 40.05
    # chirps_minlon = -20.05
    # chirps_maxlon = 55.05
    chirps_ulx      = -20.05
    chirps_uly      = 40.05
    chirps_xres_deg = 0.1
    chirps_yres_deg = -0.1     ###########   NEED NEGATIVE HERE
    chirps_ncols    = 751
    chirps_nrows    = 801
    chirps_dtype    = 'float32'
    chirps_nodata   = -1.0

In [4]:
rts_file = chirps_dir + '__CHIRPS_Rain_Baro_2015-10_to_2018-10.rts'

resample_algo = 'bilinear'
## resample_algo = 'nearest'
time_interval_hours = 6.0      #### For the 6-hourly rainfall product

os.chdir( chirps_dir )
regrid.create_rts_from_chirps_files( rts_file=rts_file,
           time_interval_hours=time_interval_hours,
           resample_algo=resample_algo,
           VERBOSE=False, DEM_bounds=DEM_bounds,
           DEM_xres_sec=DEM_xres_sec, DEM_yres_sec=DEM_yres_sec,
           DEM_ncols=DEM_ncols, DEM_nrows=DEM_nrows )


Working...

chirps_file = rfe_gdas.bin.2015100100
  resaving as GeoTIFF...
  comparing bounds...
  regridding to DEM grid...
count = 1

chirps_file = rfe_gdas.bin.2015100106
  resaving as GeoTIFF...
  comparing bounds...
  regridding to DEM grid...
count = 2

chirps_file = rfe_gdas.bin.2015100112
  resaving as GeoTIFF...
  comparing bounds...
  regridding to DEM grid...
count = 3

chirps_file = rfe_gdas.bin.2015100118
  resaving as GeoTIFF...
  comparing bounds...
  regridding to DEM grid...
count = 4

chirps_file = rfe_gdas.bin.2015100200
  resaving as GeoTIFF...
  comparing bounds...
  regridding to DEM grid...
count = 5

chirps_file = rfe_gdas.bin.2015100206
  resaving as GeoTIFF...
  comparing bounds...
  regridding to DEM grid...
count = 6

chirps_file = rfe_gdas.bin.2015100212
  resaving as GeoTIFF...
  comparing bounds...
  regridding to DEM grid...
count = 7

chirps_file = rfe_gdas.bin.2015100218
  resaving as GeoTIFF...
  comparing bounds...
  regridding to DEM grid...
count =

## CHIRPS Grid Stack for 2015-10 to 2020-06

In [ ]:
rts_file = chirps_dir2 + '__CHIRPS_Rain_Baro_2015-10_to_2020-06.rts'

resample_algo = 'bilinear'
## resample_algo = 'nearest'
time_interval_hours = 6.0      #### For the 6-hourly rainfall product

os.chdir( chirps_dir2 )
regrid.create_rts_from_chirps_files( rts_file=rts_file,
           time_interval_hours=time_interval_hours,
           resample_algo=resample_algo,
           VERBOSE=False, DEM_bounds=DEM_bounds,
           DEM_xres_sec=DEM_xres_sec, DEM_yres_sec=DEM_yres_sec,
           DEM_ncols=DEM_ncols, DEM_nrows=DEM_nrows )